# Imports

In [247]:
#!pip install -U scikit-learn

In [248]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx

# Input

In [249]:
transformation = Transformation()

user_number_points = 100

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [250]:
print(graph._node)

{(75.0252, -0.66553295, 3.8): {'index_triangle': {0, 6820, 164, 165, 6821, 6822}}, (75.01418, -0.49935842, 16.7): {'index_triangle': {0, 1, 165, 370, 371, 372}}, (75.0, 0.0, 3.8): {'index_triangle': {0, 1, 2, 6658, 6659, 11415, 11416, 11417, 6822, 6823, 6824, 6825, 6826, 6827, 6828, 6829, 6830, 6831, 6832, 6833, 6834, 6835, 6836, 6837, 6838, 6839, 6840, 6841, 6842, 6843, 6844, 6845, 6846, 6847, 6848, 6849, 6850, 6851, 6852, 6853, 6854, 6855, 6856, 6857, 6858, 6859, 6860, 6861, 6862, 6863, 6864, 6865, 6866, 6867, 6868, 6869, 6870, 6871, 6872, 6873, 6874, 6875, 6876, 6877, 6878, 6495}}, (75.00158, 0.16653232, 16.7): {'index_triangle': {1, 2, 3, 372, 373, 374}}, (75.0252, 0.66553295, 3.8): {'index_triangle': {2, 6659, 3, 4, 6660}}, (75.03937, 0.8314692, 16.7): {'index_triangle': {3, 4, 5, 374, 375, 376}}, (75.10067, 1.3272538, 3.8): {'index_triangle': {6660, 6661, 4, 5, 6, 6662}}, (75.12734, 1.4916434, 16.7): {'index_triangle': {5, 6, 7, 376, 377, 378}}, (75.22596, 1.9813722, 3.8): {'inde

# Point Sampler

### DevConv

In [251]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [252]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = np.subtract(current_node, neigh_nodes)                                               # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [253]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[0.27442453 0.27458419 0.27442453 ... 0.25548    0.37085003 0.25548   ]
[0.43994779 0.44020375 0.43994779 ... 0.40957658 0.59453377 0.40957657]
[0.68534006 0.68543772 0.68534006 ... 0.67363791 0.7411472  0.67363791]
(5999,)


### Multinomial Sampling

In [254]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[0 0 0 ... 0 0 0]


In [255]:
target_number_point = min(len(graph._node), user_number_points)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]
list_k_nodes = np.array(list(graph._node.keys()))[index_k_nodes]
list_k_nodes = [tuple(e) for e in list_k_nodes]
print(list_k_nodes)

[(88.219635, -6.7292814, -2.0857615), (81.51755, -8.610001, 1.2981478), (78.31546, 7.8332825, 93.57346), (79.10347, -8.610001, -1.8479408), (75.504074, 8.760632, 90.81827), (78.16942, -8.610001, -2.4203293), (80.67585, -8.722893, -1.3105594), (75.0, -2.3044074, 98.49292), (91.5333, 4.1574683, 2.9335263), (92.07089, 2.9401166, 2.9053526), (82.9645, -0.99710757, -11.83122), (91.75591, -3.5589852, 82.96112), (92.418976, 1.6554987, 2.8871102), (51.0, 4.0099483, 2.8332825), (78.1065, -8.722893, -3.177305), (82.89438, -8.78582, 1.2349607), (80.554214, -8.78582, -2.368579), (78.76841, -8.78582, -3.5959258), (77.56504, -8.78582, -4.0943794), (79.49034, -7.6759295, 3.5646713), (75.39972, -8.722893, 88.82714), (83.52768, -8.798424, 1.0291891), (75.891014, -3.8609922, 3.753304), (81.66343, -8.798424, -2.1997812), (81.340294, -8.798424, -2.5402958), (75.10053, -1.3272538, 3.7947316), (78.64702, -8.798424, -4.391335), (75.177315, -6.9390492, 84.583374), (77.32071, -8.798424, -4.8610053), (76.86425,

# KNN

In [256]:
XYZ = [list(e) for e in list_k_nodes]
print(XYZ)

[[88.219635, -6.7292814, -2.0857615], [81.51755, -8.610001, 1.2981478], [78.31546, 7.8332825, 93.57346], [79.10347, -8.610001, -1.8479408], [75.504074, 8.760632, 90.81827], [78.16942, -8.610001, -2.4203293], [80.67585, -8.722893, -1.3105594], [75.0, -2.3044074, 98.49292], [91.5333, 4.1574683, 2.9335263], [92.07089, 2.9401166, 2.9053526], [82.9645, -0.99710757, -11.83122], [91.75591, -3.5589852, 82.96112], [92.418976, 1.6554987, 2.8871102], [51.0, 4.0099483, 2.8332825], [78.1065, -8.722893, -3.177305], [82.89438, -8.78582, 1.2349607], [80.554214, -8.78582, -2.368579], [78.76841, -8.78582, -3.5959258], [77.56504, -8.78582, -4.0943794], [79.49034, -7.6759295, 3.5646713], [75.39972, -8.722893, 88.82714], [83.52768, -8.798424, 1.0291891], [75.891014, -3.8609922, 3.753304], [81.66343, -8.798424, -2.1997812], [81.340294, -8.798424, -2.5402958], [75.10053, -1.3272538, 3.7947316], [78.64702, -8.798424, -4.391335], [75.177315, -6.9390492, 84.583374], [77.32071, -8.798424, -4.8610053], [76.86425,

In [257]:
def connect_extended_graph(XYZ, number_neigh=15):
    # Create the nearest neighbors object
    _, indices = NearestNeighbors(n_neighbors=number_neigh).fit(XYZ).kneighbors(XYZ)
    # Create the graph from the nearest points
    extended_graph = nx.Graph()
    for index_poly, poly in enumerate(indices):
        for index_current_node in range(len(poly)):
            current_node = tuple(XYZ[poly[index_current_node]])
            for index_other_node in range(index_current_node+1, len(poly)):
                edge = current_node, tuple(XYZ[poly[index_other_node]])
                extended_graph.add_edge(*edge)
                # if attribute do not exists
                if len(extended_graph.nodes[current_node])==0:
                    extended_graph.nodes[current_node]['index_poly'] = set()
                extended_graph.nodes[current_node]['index_poly'].add(index_poly)
    return extended_graph

In [258]:
extended_graph = connect_extended_graph(XYZ)

In [259]:
transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 100
Number of edges: 1407


# Edge Predictor

In [260]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(100, 64)

In [261]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""
from numpy import mean

f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [262]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

# Face Candidates

#### Inputs

In [263]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 18)	1
  (0, 19)	1
  (0, 20)	1
  (0, 40)	1
  (0, 43)	1
  (0, 52)	1
  (0, 53)	1
  (0, 54)	1
  (0, 57)	1
  :	:
  (98, 88)	1
  (98, 89)	1
  (98, 90)	1
  (98, 91)	1
  (98, 92)	1
  (98, 93)	1
  (98, 94)	1
  (98, 97)	1
  (98, 99)	1
  (99, 49)	1
  (99, 50)	1
  (99, 51)	1
  (99, 84)	1
  (99, 85)	1
  (99, 86)	1
  (99, 87)	1
  (99, 88)	1
  (99, 89)	1
  (99, 90)	1
  (99, 91)	1
  (99, 92)	1
  (99, 93)	1
  (99, 94)	1
  (99, 97)	1
  (99, 98)	1
[[2.58114332e-03 2.51161695e-03 1.96473152e-03 ... 1.06469642e-06
  1.23084466e-06 6.06023980e-05]
 [4.57328111e+05 1.93319389e-03 1.47906044e-03 ... 2.86891648e-07
  3.40243082e-07 3.08864853e-05]
 [9.02283302e+05 1.47919189e-01 3.10079528e-03 ... 2.23301819e-06
  2.58367392e-06 1.20101897e-04]
 ...
 [2.88779200e+17 7.69018151e+17 1.41373068e+18 ... 8.70676915e-45
  2.895718

In [264]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0

A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T

A_s = A_s/A_s.max()                                         # Normalize

In [265]:
print(A_s)  # symmétrique
print(A_s.shape)

[[9.65022538e-271 6.06867979e-264 1.19731746e-263 ... 2.01019281e-186
  1.99065214e-187 1.11539920e-214]
 [6.06867979e-264 9.64528189e-264 3.13602186e-263 ... 1.31027241e-186
  1.29754125e-187 3.87866323e-213]
 [1.19731746e-263 3.13602186e-263 8.61996024e-263 ... 3.56655962e-186
  3.53187618e-187 7.70681705e-213]
 ...
 [2.01019281e-186 1.31027241e-186 3.56655962e-186 ... 1.59087476e-100
  1.57519348e-101 8.80890121e-129]
 [1.99065214e-187 1.29754125e-187 3.53187618e-187 ... 1.57519348e-101
  1.55961761e-102 8.71559629e-130]
 [1.11539920e-214 3.87866323e-213 7.70681705e-213 ... 8.80890121e-129
  8.71559629e-130 4.79183615e-157]]
(100, 100)


# Face Classifier

### TriConv

#### Inputs

In [266]:
triangles = list(nx.simple_cycles(extended_graph, length_bound=3))  # [triangle0, triangle1,...] | triangle0 = [node1,node2,node3] | node1 = (x ,y ,z)
# triangles = cycles
print(triangles[0])
print(np.array(triangles).shape)    #nb_triangle, 3 nodes, 3 dimensions par node

[(88.219635, -6.7292814, -2.0857615), (88.01927, -7.138877, -1.1976296), (86.959114, -7.8332825, -0.79067487)]
(9507, 3, 3)


In [267]:
p_init = np.zeros((len(triangles)))

for index_triangle, triangle in enumerate(triangles):
    i = list(dict(extended_graph._node).keys()).index(triangle[0])
    j = list(dict(extended_graph._node).keys()).index(triangle[1])
    k = list(dict(extended_graph._node).keys()).index(triangle[2])
    p_init[index_triangle] = (A_s[i,j] + A_s[i,k] + A_s[j,k])/3
print(p_init)
print(len(p_init))

[1.64673576e-263 1.19194577e-263 3.64599135e-262 ... 8.72410842e-261
 2.70966337e-260 2.41008745e-260]
9507


#### Calculate barycenter

In [268]:
barycenters = list()

for _, triangle in enumerate(triangles):
    b_x = (triangle[0][0] + triangle[1][0] + triangle[2][0]) / 3
    b_y = (triangle[0][1] + triangle[1][1] + triangle[2][1]) / 3
    b_z = (triangle[0][2] + triangle[1][2] + triangle[2][2]) / 3
    barycenters.append([b_x, b_y, b_z])

print(barycenters[0])
print(len(barycenters))

[87.73267618815105, -7.233813603719075, -1.358021895090739]
9507


#### KNN Tri

In [279]:
graph_tri = connect_extended_graph(barycenters, number_neigh=20)

In [283]:
transformation.print_graph_properties(graph)
print("")
transformation.print_graph_properties(extended_graph)
print("")
transformation.print_graph_properties(graph_tri)

Number of nodes: 5999
Number of edges: 17991

Number of nodes: 100
Number of edges: 1407

Number of nodes: 9507
Number of edges: 258079


#### calculate e norm matrix

In [282]:
diff_vectors = list()

for _, triangle in enumerate(triangles):
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])
print(diff_vectors)
print(len(diff_vectors))

[[0.9983448, 2.1177773, 1.3310694], [0.9983448, 2.5579545, 1.5778352], [0.9983448, 3.3757408, 3.3788059], [0.9983448, 3.8426542, 2.8580198], [0.9983448, 3.884553, 3.755776], [0.9983448, 4.0171385, 3.3110015], [0.9983448, 4.4941416, 3.640513], [0.9983448, 5.2315764, 4.2490144], [0.9983448, 5.2452517, 5.7212863], [0.9983448, 5.586091, 4.621993], [0.9983448, 5.984913, 5.0481825], [0.9983448, 5.9998927, 5.280835], [0.9983448, 6.604158, 5.907152], [0.9983448, 12.437642, 12.530852], [0.9983448, 11.542992, 11.611899], [0.9983448, 10.614539, 10.64817], [0.9983448, 7.7398915, 7.1179724], [0.9983448, 6.875913, 6.644933], [0.9983448, 7.1981444, 7.0118184], [0.9983448, 7.941537, 7.7337327], [0.9983448, 7.841181, 7.513169], [0.9983448, 7.37389, 7.674622], [0.9983448, 9.886686, 9.988835], [0.9983448, 10.736977, 11.136364], [0.9983448, 10.061412, 10.039571], [2.1177773, 2.5579545, 1.524127], [2.1177773, 3.3757408, 2.6976275], [2.1177773, 3.8426542, 2.0721104], [2.1177773, 3.884553, 2.8733802], [2.117

#### Calculate r

In [270]:
r_matrix = np.zeros((len(triangles),len(triangles), 5))

max_diff_vectors = np.array(diff_vectors).max(axis=1)
min_diff_vectors = np.array(diff_vectors).min(axis=1)

barycenters_numpy = np.array(barycenters)

# Calculate differences once to avoid redundant computations
diff_vectors = min_diff_vectors[:, np.newaxis] - min_diff_vectors
max_diff_vectors_diff = max_diff_vectors[:, np.newaxis] - max_diff_vectors
barycenters_diff = barycenters_numpy[:, np.newaxis, :] - barycenters_numpy

# Populate the r_matrix using vectorized operations
r_matrix = np.stack([diff_vectors, max_diff_vectors_diff, barycenters_diff[:, :, 0], barycenters_diff[:, :, 1], barycenters_diff[:, :, 2]], axis=-1)

print(r_matrix.shape)   # nb_triangles, nb_triangles, 5dim/triangles

(9507, 9507, 5)


#### Calculate f

In [271]:
# MLP * 3 
f_final = p_init    # TODO

import torch
final_scores = torch.nn.functional.softmax(torch.tensor(f_final))
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

0.9999999999999678
(9507,)
[0.00010495 0.00010495 0.00010495 ... 0.00010495 0.00010495 0.00010495]


C:\Users\Arthur\AppData\Local\Temp\ipykernel_14912\4023725568.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))


# Simplified Mesh

In [272]:
selected_triangles_indexes = np.argpartition(final_scores, -target_number_point)[-target_number_point:] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)

(100, 3, 3)


In [273]:
def create_triangle_graph(XYZ):
    extended_graph = nx.Graph()
    for index_poly, poly in enumerate(XYZ):
        for index_current_node in range(len(poly)):
            current_node = tuple(poly[index_current_node])
            for index_other_node in range(index_current_node+1, len(poly)):
                edge = current_node, tuple(poly[index_other_node])
                extended_graph.add_edge(*edge)
                # if attribute do not exists
                if len(extended_graph.nodes[current_node])==0:
                    extended_graph.nodes[current_node]['index_triangle'] = set()
                extended_graph.nodes[current_node]['index_triangle'].add(index_poly)
                if len(extended_graph.nodes[tuple(poly[index_other_node])])==0:
                    extended_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
                extended_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
    return extended_graph

In [274]:
simplified_final_graph = create_triangle_graph(selected_triangles)
print(simplified_final_graph._node)

{(75.60927, -3.2518802, 81.329506): {'index_triangle': {0}}, (75.011444, 0.66553295, 81.22246): {'index_triangle': {0, 1, 5, 8, 10, 21, 39, 40, 43, 44, 45, 47, 54, 55, 56, 57, 58, 59, 60, 61, 65, 77, 78, 79, 80, 84, 87, 92, 93, 94, 97}}, (39.0, -5.921278, 83.49013): {'index_triangle': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 21}}, (63.0, -6.3966537, 83.95659): {'index_triangle': {1}}, (39.0, -1.8187612, 81.39): {'index_triangle': {2, 3, 4, 5}}, (75.02245, 0.66553295, 81.21144): {'index_triangle': {2, 6, 9, 10, 11, 17, 20, 25, 26, 29, 32, 34, 35, 41, 50, 52, 53, 60, 62, 63, 68, 70, 71, 74, 82, 84, 86, 89, 96}}, (75.02397, 0.66553295, 81.20779): {'index_triangle': {3, 6, 7, 8, 12, 15, 18, 23, 30, 33, 36, 37, 38, 48, 51, 57, 58, 64, 66, 69, 70, 71, 72, 73, 75, 83, 90, 91, 98}}, (75.013725, 0.66553295, 81.22114): {'index_triangle': {4, 7, 9, 13, 14, 16, 19, 21, 22, 24, 27, 28, 31, 42, 46, 49, 52, 53, 54, 55, 64, 66, 67, 76, 81, 85, 88, 95, 99}}, (75.054825, 1.3272538, 3.7155738): {'index_triangl

In [275]:
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 19
Number of edges: 82


In [276]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)